In [1]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import os
from PIL import Image
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple

import torchvision
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.models 
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torchvision.transforms import functional as F
import torchvision.models as models

In [2]:
test_df = pd.read_csv('../input/global-wheat-detection/' + 'sample_submission.csv')
test_path = '../input/global-wheat-detection/test'

In [3]:
class global_wheat_detection(Data.Dataset):
    def __init__(self, dataframe, image_dir, transform = None):

        self.transform = transform
        self.df = dataframe
        self.image_dir = image_dir
        self.image_ids = dataframe['image_id'].unique()
        


    def __getitem__(self, index):
        image_id = self.image_ids[index]
        
        image_path = os.path.join(self.image_dir, image_id + '.jpg')
        img = np.array(Image.open(image_path).convert("RGB"))
        
        
        img = T.ToTensor()(img)
            

        return img, image_id
            
    def __len__(self):

        return self.image_ids.shape[0]



In [4]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
model.load_state_dict(torch.load('../input/modelwieght2/17fasterrcnn_resnet50_fpn.pth'))

model.to(device)
model.eval()
a = 1

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = global_wheat_detection(test_df, test_path)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [9]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [10]:
detection_threshold = 0.5
results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        
        results.append(result)

In [11]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

,image_id,PredictionString
0,aac893a91,0.9950 69 0 102 161 0.9923 617 912 76 111 0.98...
1,51f1be19e,0.9948 605 82 157 176 0.9873 276 471 135 123 0...
2,f5a1f0358,0.9931 552 404 75 93 0.9924 546 271 106 110 0....
3,796707dd7,0.9892 713 824 105 100 0.9887 901 335 97 86 0....
4,51b3e36ab,0.9980 836 451 183 155 0.9978 0 436 101 315 0....


In [12]:
test_df.to_csv('submission.csv', index=False)